In [1]:
from openai import OpenAI
from pathlib import Path

key = open('openai.key').read().strip()
client = OpenAI(api_key=key, organization='org-l0QXnTWrsY221IPu8QIF1k1H', project='proj_hdXOnDa32fjKeT330EowFZsc')

In [2]:
txt_path = Path('public_agenda_june24_.txt')

In [3]:
assistant = client.beta.assistants.create(
    name="Technology Purchases",
    instructions="You are a research assistant. Your task is to list any mentions of technology purchases in the document.",
    model='gpt-4o',
)

In [5]:
# Read 10000 characters from the text file at a time with overlap of 100
with txt_path.open() as f:
    text = f.read()
    
    for i in range(0, len(text), 9000):
        chunk = text[i:i+10000]

        thread = client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    "content": "list any mentions of technology purchases in the following: " + chunk,
                }
            ]
        )
    
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id=assistant.id
        )
    
        messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
    
        message_content = messages[0].content[0].text
        
        print(message_content.value)
        
        break

Here are the mentions of technology purchases from the provided document:

1. **May 21, 2024 from Myron Demkiw, Chief of Police Re: Contract Extension and Increase with Versaterm Public Safety Inc. for the Versadex Records Management System**.
